In [2]:
pip install rioxarray rasterio pystac_client planetary_computer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 79.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 13.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Data Science
import numpy as np
import pandas as pd
# Multi-dimensional arrays and datasets
import xarray as xr
# Geospatial raster data handling
import rioxarray as rxr
# Geospatial data analysis
import geopandas as gpd
# Geospatial operations
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
from rasterio.warp import transform_bounds
from rasterio.windows import from_bounds
# Image Processing
from PIL import Image
# Coordinate transformations
from pyproj import Proj, Transformer, CRS
# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo
# Others
import os
from tqdm import tqdm

In [5]:
ground_df = pd.read_csv("/kaggle/input/ey-challenge/Training_data_uhi_index_2025-02-18.csv")
ground_df.head()

,Longitude,Latitude,datetime,UHI Index
0,-73.909167,40.813107,24-07-2021 15:53,1.030289
1,-73.909187,40.813045,24-07-2021 15:53,1.030289
2,-73.909215,40.812978,24-07-2021 15:53,1.023798
3,-73.909242,40.812908,24-07-2021 15:53,1.023798
4,-73.909257,40.812845,24-07-2021 15:53,1.021634


In [ ]:
# tiff_path = "S2_sample.tiff"

# with rasterio.open(tiff_path) as src1:
#     band_b01 = src1.read(1)
#     band_b02 = src1.read(2)
#     band_b03 = src1.read(3)
#     band_b04 = src1.read(4)
#     band_b05 = src1.read(5)
#     band_b06 = src1.read(6)
#     band_b07 = src1.read(7)
#     band_b08 = src1.read(8)
#     band_b8a = src1.read(9)
#     band_b11 = src1.read(10)
#     band_b12 = src1.read(11)


In [11]:
def map_satellite_data(tiff_path, csv_path):
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values
    
    proj_wgs84 = Proj(init='epsg:4326')
    proj_tiff = Proj(tiff_crs)
    
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)
    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []
    
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
        B01_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        B01_values.append(B01_value)
        
        B02_value = data.sel(x=lon, y=lat, band=2, method="nearest").values
        B02_values.append(B02_value)
        
        B03_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
        B03_values.append(B03_value)
        
        B04_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B04_values.append(B04_value)
        
        B05_value = data.sel(x=lon, y=lat, band=5, method="nearest").values
        B05_values.append(B05_value)
        
        B06_value = data.sel(x=lon, y=lat, band=6, method="nearest").values
        B06_values.append(B06_value)
        
        B07_value = data.sel(x=lon, y=lat, band=7, method="nearest").values
        B07_values.append(B07_value)
        
        B08_value = data.sel(x=lon, y=lat, band=8, method="nearest").values
        B08_values.append(B08_value)
        
        B8A_value = data.sel(x=lon, y=lat, band=8, method="nearest").values
        B8A_values.append(B8A_value)
        
        B11_value = data.sel(x=lon, y=lat, band=11, method="nearest").values
        B11_values.append(B11_value)
        
        B12_value = data.sel(x=lon, y=lat, band=12, method="nearest").values
        B12_values.append(B12_value)
        
    df = pd.DataFrame()
    df['B01'] = B01_values
    df['B02'] = B02_values
    df['B03'] = B03_values
    df['B04'] = B04_values
    df['B05'] = B05_values
    df['B06'] = B06_values
    df['B07'] = B07_values
    df['B08'] = B08_values
    df['B8A'] = B8A_values
    df['B11'] = B11_values
    df['B12'] = B12_values
    return df

In [12]:
final_data = map_satellite_data('/kaggle/input/ey-challenge/S2_sample.tiff', '/kaggle/input/ey-challenge/Training_data_uhi_index_2025-02-18.csv')
final_data

Mapping values: 100%|██████████| 11229/11229 [02:58<00:00, 62.75it/s]


,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12
0,846.0,1042.0,1036.0,1036.0,1272.0,1502.0,1605.0,1906.0,1906.0,1265.0,1206.0
1,846.0,1042.0,1036.0,1036.0,1272.0,1502.0,1605.0,1906.0,1906.0,1265.0,1206.0
2,846.0,583.0,818.0,709.0,1054.0,1668.0,2097.0,2190.0,2190.0,991.0,777.0
3,846.0,581.0,733.0,657.0,1054.0,1668.0,2097.0,2182.0,2182.0,991.0,741.5
4,846.0,655.0,744.0,745.0,1021.0,1728.0,1943.0,2112.0,2112.0,1134.0,708.5
...,...,...,...,...,...,...,...,...,...,...,...
11224,481.0,473.0,708.0,528.0,990.0,2382.0,2494.0,3284.0,3284.0,1079.0,501.0
11225,481.0,540.0,742.0,610.0,990.0,2382.0,2494.0,2900.0,2900.0,1079.0,551.5
11226,481.0,540.0,742.0,610.0,990.0,2382.0,2494.0,2900.0,2900.0,1079.0,551.5
11227,481.0,540.0,742.0,610.0,990.0,2382.0,2494.0,2900.0,2900.0,1079.0,551.5


In [13]:
final_data['NDVI'] = (final_data['B08'] - final_data['B04']) / (final_data['B08'] + final_data['B04'])
final_data['NDVI'] = final_data['NDVI'].replace([np.inf, -np.inf], np.nan)

final_data['NDBI'] = (final_data['B11'] - final_data['B08']) / (final_data['B11'] + final_data['B08'])
final_data['NDBI'] = final_data['NDBI'].replace([np.inf, -np.inf], np.nan)

final_data['NDWI'] = (final_data['B03'] - final_data['B08']) / (final_data['B03'] + final_data['B08'])
final_data['NDWI'] = final_data['NDWI'].replace([np.inf, -np.inf], np.nan)

In [24]:
uhi_data = pd.concat([ground_df,final_data], axis=1)
uhi_data

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,846.0,1042.0,1036.0,1036.0,1272.0,1502.0,1605.0,1906.0,1906.0,1265.0,1206.0,0.295717,-0.202144,-0.295717
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,846.0,1042.0,1036.0,1036.0,1272.0,1502.0,1605.0,1906.0,1906.0,1265.0,1206.0,0.295717,-0.202144,-0.295717
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,846.0,583.0,818.0,709.0,1054.0,1668.0,2097.0,2190.0,2190.0,991.0,777.0,0.510866,-0.376925,-0.456117
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,846.0,581.0,733.0,657.0,1054.0,1668.0,2097.0,2182.0,2182.0,991.0,741.5,0.537161,-0.375355,-0.497084
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,846.0,655.0,744.0,745.0,1021.0,1728.0,1943.0,2112.0,2112.0,1134.0,708.5,0.478474,-0.301294,-0.478992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,24-07-2021 15:57,0.972470,481.0,473.0,708.0,528.0,990.0,2382.0,2494.0,3284.0,3284.0,1079.0,501.0,0.722980,-0.505386,-0.645291
11225,-73.957063,40.790308,24-07-2021 15:57,0.972470,481.0,540.0,742.0,610.0,990.0,2382.0,2494.0,2900.0,2900.0,1079.0,551.5,0.652422,-0.457653,-0.592532
11226,-73.957093,40.790270,24-07-2021 15:57,0.981124,481.0,540.0,742.0,610.0,990.0,2382.0,2494.0,2900.0,2900.0,1079.0,551.5,0.652422,-0.457653,-0.592532
11227,-73.957112,40.790253,24-07-2021 15:59,0.981245,481.0,540.0,742.0,610.0,990.0,2382.0,2494.0,2900.0,2900.0,1079.0,551.5,0.652422,-0.457653,-0.592532


In [25]:
# columns_to_check = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08","B8A", "B11", "B12", 'NDVI', 'NDBI', 'NDWI']
# for col in columns_to_check:
#     uhi_data[col] = uhi_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# uhi_data = uhi_data.drop_duplicates(subset=columns_to_check, keep='first')
# uhi_data = uhi_data.reset_index(drop=True)
# uhi_data

In [28]:
correlation_with_uhi = uhi_data.drop(columns=['datetime', 'Longitude', 'Latitude']).corr()['UHI Index']
correlation_with_uhi

UHI Index    1.000000
B01          0.193179
B02          0.163307
B03          0.168818
B04          0.175350
B05          0.170996
B06          0.117021
B07          0.094459
B08          0.082083
B8A          0.082083
B11          0.187564
B12          0.149961
NDVI        -0.254888
NDBI         0.187625
NDWI         0.250060
Name: UHI Index, dtype: float64

In [29]:
uhi_data = uhi_data[['B01', 'B02', 'B03', 'B04', 'B05', 'B11', 'B12', 'NDVI', 'NDBI', 'NDWI', 'UHI Index']]

X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [30]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [31]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [32]:
insample_predictions = model.predict(X_train)
Y_train = y_train.tolist()
r2_score(Y_train, insample_predictions)

0.9398321417945676

In [33]:
outsample_predictions = model.predict(X_test)
Y_test = y_test.tolist()
r2_score(Y_test, outsample_predictions)

0.5971820005837326

In [34]:
test_file = pd.read_csv('/kaggle/input/ey-challenge/Submission_template_UHI2025-v2.csv')
test_file.head()

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,NaN
1,-73.971928,40.788875,NaN
2,-73.967080,40.789080,NaN
3,-73.972550,40.789082,NaN
4,-73.969697,40.787953,NaN


In [35]:
val_data = map_satellite_data('/kaggle/input/ey-challenge/S2_sample.tiff', '/kaggle/input/ey-challenge/Submission_template_UHI2025-v2.csv')

Mapping values: 100%|██████████| 1040/1040 [00:16<00:00, 61.56it/s]


In [36]:
val_data['NDVI'] = (val_data['B08'] - val_data['B04']) / (val_data['B08'] + val_data['B04'])
val_data['NDVI'] = val_data['NDVI'].replace([np.inf, -np.inf], np.nan)

val_data['NDBI'] = (val_data['B11'] - val_data['B08']) / (val_data['B11'] + val_data['B08'])
val_data['NDBI'] = val_data['NDBI'].replace([np.inf, -np.inf], np.nan)

val_data['NDWI'] = (val_data['B03'] - val_data['B08']) / (val_data['B03'] + val_data['B08'])
val_data['NDWI'] = val_data['NDWI'].replace([np.inf, -np.inf], np.nan)

In [37]:
submission_val_data = val_data.loc[:,['B01', 'B02', 'B03', 'B04', 'B05', 'B11', 'B12', 'NDVI', 'NDBI', 'NDWI']]
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [38]:
final_predictions = model.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

submission_df = pd.DataFrame({'Longitude':test_file['Longitude'].values, 'Latitude':test_file['Latitude'].values, 'UHI Index':final_prediction_series.values})
submission_df

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,0.978034
1,-73.971928,40.788875,0.977254
2,-73.967080,40.789080,0.998300
3,-73.972550,40.789082,0.998791
4,-73.969697,40.787953,0.971390
...,...,...,...
1035,-73.919388,40.813803,1.010172
1036,-73.931033,40.833178,1.022960
1037,-73.934647,40.854542,1.009379
1038,-73.917223,40.815413,1.000510


In [39]:
submission_df.to_csv("submission.csv",index = False)